## Setup and Command Spot to Move

#### Create objects

All Boston Dynamics API programs start by creating an SDK object with a client name argument. The client name is used to help with debugging, and does not have any semantic information - so use whatever string is helpful for you.

In [2]:
import bosdyn.client

sdk = bosdyn.client.create_standard_sdk('understanding-spot')

To retrieve the robot id like we did in Spot Python SDK Quickstart we’ll first need to create a robot object, using its network address as an argument. In this example, we only create one robot object, but it is possible to create and control multiple robots in the same program with the Boston Dynamics API.

In [3]:
robot = sdk.create_robot('192.168.80.3')

#### Services and Authentication

-- Need to be on the Spot Wifi for this step --

Before robot state can be retrieved, you need to authenticate to the robot. The majority of services require the user to be authenticated - this prevents random network attackers from being able to control the robot or intercept information which might be sensitive.

In [5]:
robot.authenticate('user', 'hhe262iz9zez')

Obtain information about Spot:

In [6]:
state_client = robot.ensure_client('robot-state')

In [18]:
state_client.get_robot_state()

power_state {
  motor_power_state: STATE_OFF
  shore_power_state: STATE_OFF_SHORE_POWER
  robot_power_state: ROBOT_POWER_STATE_ON
  payload_ports_power_state: PAYLOAD_PORTS_POWER_STATE_ON
  wifi_radio_power_state: WIFI_RADIO_POWER_STATE_ON
  locomotion_charge_percentage {
    value: 16
  }
  locomotion_estimated_runtime {
    seconds: 948
    nanos: 557373046
  }
}
battery_states {
  identifier: "a2-22b14-001e"
  charge_percentage {
    value: 16
  }
  estimated_runtime {
    seconds: 948
    nanos: 557373046
  }
  current {
    value: -4.1600003242492676
  }
  voltage {
    value: 47.6140022277832
  }
  temperatures: 36.437038421630859
  temperatures: 36.144443511962891
  temperatures: 36.181480407714844
  temperatures: 35.911109924316406
  temperatures: 35.374073028564453
  temperatures: 36.355556488037109
  temperatures: 35.548149108886719
  temperatures: 35.629631042480469
  temperatures: 35.962963104248047
  temperatures: 35.974075317382812
  status: STATUS_DISCHARGING
}
comms_sta

### Capture and View Camera Images

In [7]:
from bosdyn.client.image import ImageClient

image_client = robot.ensure_client(ImageClient.default_service_name)
sources = image_client.list_image_sources()
[source.name for source in sources]

['back_depth',
 'back_depth_in_visual_frame',
 'back_fisheye_image',
 'frontleft_depth',
 'frontleft_depth_in_visual_frame',
 'frontleft_fisheye_image',
 'frontright_depth',
 'frontright_depth_in_visual_frame',
 'frontright_fisheye_image',
 'hand_color_image',
 'hand_color_in_hand_depth_frame',
 'hand_depth',
 'hand_depth_in_hand_color_frame',
 'hand_image',
 'left_depth',
 'left_depth_in_visual_frame',
 'left_fisheye_image',
 'right_depth',
 'right_depth_in_visual_frame',
 'right_fisheye_image']

In [30]:
image_response = image_client.get_image_from_sources(["hand_color_image"])[0] # see options for camera aboove 
from PIL import Image
import io
image = Image.open(io.BytesIO(image_response.shot.image.data))
image.show()

### Configuring “Motor Power Authority” (software E-Stop)

In [9]:
# creating a client to the E-Stop service and requesting status
estop_client = robot.ensure_client('estop')
estop_client.get_status()

stop_level: ESTOP_LEVEL_CUT
stop_level_details: "Not all endpoints are registered"

In [22]:
# create and register an E-Stop Endpoint
estop_endpoint = bosdyn.client.estop.EstopEndpoint(client=estop_client, name='my_estop', estop_timeout=9.0)
estop_endpoint.force_simple_setup()
estop_client.get_status()

endpoints {
  endpoint {
    role: "PDB_rooted"
    name: "my_estop"
    unique_id: "5"
    timeout {
      seconds: 9
    }
    cut_power_timeout {
      seconds: 17
    }
  }
  stop_level: ESTOP_LEVEL_CUT
  time_since_valid_response {
  }
}
stop_level: ESTOP_LEVEL_CUT
stop_level_details: "Endpoint requested stop"

In [23]:
# clear e-stop
estop_keep_alive = bosdyn.client.estop.EstopKeepAlive(estop_endpoint)
estop_client.get_status()

endpoints {
  endpoint {
    role: "PDB_rooted"
    name: "my_estop"
    unique_id: "5"
    timeout {
      seconds: 9
    }
    cut_power_timeout {
      seconds: 17
    }
  }
  stop_level: ESTOP_LEVEL_NONE
  time_since_valid_response {
    nanos: 3000064
  }
}
stop_level: ESTOP_LEVEL_NONE

bosdyn.api.EstopCheckInResponse (EndpointUnknownError): The endpoint specified in the request is not registered.


### Taking ownership of Spot (Leases)

In [10]:
lease_client = robot.ensure_client('lease')
lease_client.list_leases()

[resource: "all-leases"
lease {
  resource: "all-leases"
  epoch: "KsxuRazVOfrwXobi"
  sequence: 0
  client_names: "root"
}
lease_owner {
}
, resource: "arm"
lease {
  resource: "arm"
  epoch: "KsxuRazVOfrwXobi"
  sequence: 0
  client_names: "root"
}
lease_owner {
}
, resource: "body"
lease {
  resource: "body"
  epoch: "KsxuRazVOfrwXobi"
  sequence: 0
  client_names: "root"
}
lease_owner {
}
, resource: "fan"
lease {
  resource: "fan"
  epoch: "KsxuRazVOfrwXobi"
  sequence: 0
  client_names: "root"
}
lease_owner {
}
, resource: "full-arm"
lease {
  resource: "full-arm"
  epoch: "KsxuRazVOfrwXobi"
  sequence: 0
  client_names: "root"
}
lease_owner {
}
, resource: "gripper"
lease {
  resource: "gripper"
  epoch: "KsxuRazVOfrwXobi"
  sequence: 0
  client_names: "root"
}
lease_owner {
}
, resource: "mobility"
lease {
  resource: "mobility"
  epoch: "KsxuRazVOfrwXobi"
  sequence: 0
  client_names: "root"
}
lease_owner {
}
]

In [31]:
lease = lease_client.return_lease(lease)

Generic exception for  during check-in:
bosdyn.api.RetainLeaseResponse (LeaseUseError): 
    (resuming check-in)
Generic exception for  during check-in:
bosdyn.api.RetainLeaseResponse (LeaseUseError): 
    (resuming check-in)
Generic exception for  during check-in:
bosdyn.api.RetainLeaseResponse (LeaseUseError): 
    (resuming check-in)
Generic exception for  during check-in:
bosdyn.api.RetainLeaseResponse (LeaseUseError): 
    (resuming check-in)
Generic exception for  during check-in:
bosdyn.api.RetainLeaseResponse (LeaseUseError): 
    (resuming check-in)
Generic exception for  during check-in:
bosdyn.api.RetainLeaseResponse (LeaseUseError): 
    (resuming check-in)
Generic exception for  during check-in:
bosdyn.api.RetainLeaseResponse (LeaseUseError): 
    (resuming check-in)
Generic exception for  during check-in:
bosdyn.api.RetainLeaseResponse (LeaseUseError): 
    (resuming check-in)
Generic exception for  during check-in:
bosdyn.api.RetainLeaseResponse (LeaseUseError): 
    (re

In [11]:
lease = lease_client.acquire()
lease_keep_alive = bosdyn.client.lease.LeaseKeepAlive(lease_client)
lease_client.list_leases()

[resource: "all-leases"
lease {
  resource: "all-leases"
  epoch: "KsxuRazVOfrwXobi"
  sequence: 1
  client_names: "root"
}
lease_owner {
  client_name: "understanding-spotDK3338147W1:66208"
}
, resource: "arm"
lease {
  resource: "arm"
  epoch: "KsxuRazVOfrwXobi"
  sequence: 1
  client_names: "root"
}
lease_owner {
  client_name: "understanding-spotDK3338147W1:66208"
}
, resource: "body"
lease {
  resource: "body"
  epoch: "KsxuRazVOfrwXobi"
  sequence: 1
  client_names: "root"
}
lease_owner {
  client_name: "understanding-spotDK3338147W1:66208"
}
, resource: "fan"
lease {
  resource: "fan"
  epoch: "KsxuRazVOfrwXobi"
  sequence: 1
  client_names: "root"
}
lease_owner {
  client_name: "understanding-spotDK3338147W1:66208"
}
, resource: "full-arm"
lease {
  resource: "full-arm"
  epoch: "KsxuRazVOfrwXobi"
  sequence: 1
  client_names: "root"
}
lease_owner {
  client_name: "understanding-spotDK3338147W1:66208"
}
, resource: "gripper"
lease {
  resource: "gripper"
  epoch: "KsxuRazVOfrwX

In [14]:
robot.power_on(timeout_sec=20)
#robot.is_powered_on()

In [15]:
# timesync
robot.time_sync.wait_for_sync()

### Commanding the robot

In [23]:
from bosdyn.client.robot_command import RobotCommandClient, blocking_stand, blocking_sit, blocking_selfright
command_client = robot.ensure_client(RobotCommandClient.default_service_name)


In [28]:
# Stand
blocking_stand(command_client, timeout_sec=10)

In [47]:
# Sit
blocking_sit(command_client, timeout_sec=10)

In addition, the stand command can be modified to control the height of the body as well as the orientation of the body with respect to the footprint frame. The footprint frame is a gravity aligned frame with its origin located at the geometric center of the feet. The Z axis up, and the X axis is forward.

In [46]:
# Rotate around the z axis:

from bosdyn.geometry import EulerZXY
footprint_R_body = EulerZXY(yaw=0.5, roll=0, pitch=0)

from bosdyn.client.robot_command import RobotCommandBuilder
cmd = RobotCommandBuilder.synchro_stand_command(footprint_R_body=footprint_R_body)
command_client.robot_command(cmd)

27

### Powering off the robot

Power off the robot using the power_off command. Note the preferred method is with cut_immediately=False where Spot will come to a stop and sit down gently before powering off. The other power off option cuts motor power immediately, which causes the robot to collapse.

In [ ]:
robot.power_off(cut_immediately=False)

Unclassified exception: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Stream removed"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:192.168.80.3:443 {created_time:"2023-01-25T14:52:41.434740804+00:00", grpc_status:2, grpc_message:"Stream removed"}"
>
Generic exception for  during check-in:
RpcError: An error occurred trying to reach a service on the robot.
    (resuming check-in)
Generic exception for  during check-in:
UnableToConnectToRobotError: The robot may be offline or otherwise unreachable.
    (resuming check-in)
Generic exception for  during check-in:
UnableToConnectToRobotError: The robot may be offline or otherwise unreachable.
    (resuming check-in)
Generic exception for  during check-in:
UnableToConnectToRobotError: The robot may be offline or otherwise unreachable.
    (resuming check-in)
Generic exception for  during check-in:
UnableToConnectToRobotError: The robot may be offline or otherwise unreachable.
    (re